In [2]:
df_yield = spark.read.format("jdbc") \
                     .option("driver", "com.mysql.cj.jdbc.Driver") \
                     .option("url", "jdbc:mysql://116.204.183.76:13306/yield") \
                     .option("user", "cube_user") \
                     .option("password", "cu6euser") \
                     .option("dbtable", "yield_data") \
                     .load()

In [4]:
df_yield.summary().toPandas()

,summary,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp
0,count,28242,28242,28242,28242,28242,28242,28242
1,mean,None,None,2001.5442957297641,77053.33209404434,1149.055980454642,37076.909343529136,20.542626584519553
2,stddev,None,None,7.0519052853951205,84956.61289666739,709.8121499492227,59958.78466505776,6.312050836049751
3,min,Albania,Cassava,1990,50,51.0,0.04,1.3
4,25%,None,None,1995,19910,593.0,1702.0,16.7
5,50%,None,None,2001,38295,1083.0,17517.76,21.51
6,75%,None,None,2008,104537,1668.0,48687.88,26.0
7,max,Zimbabwe,Yams,2013,501412,3240.0,367778.0,30.65


In [5]:
df_yield.limit(10).toPandas()

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp
0,Albania,Maize,1990,36613,1485.0,121.0,16.37
1,Albania,Potatoes,1990,66667,1485.0,121.0,16.37
2,Albania,"Rice, paddy",1990,23333,1485.0,121.0,16.37
3,Albania,Sorghum,1990,12500,1485.0,121.0,16.37
4,Albania,Soybeans,1990,7000,1485.0,121.0,16.37
5,Albania,Wheat,1990,30197,1485.0,121.0,16.37
6,Albania,Maize,1991,29068,1485.0,121.0,15.36
7,Albania,Potatoes,1991,77818,1485.0,121.0,15.36
8,Albania,"Rice, paddy",1991,28538,1485.0,121.0,15.36
9,Albania,Sorghum,1991,6667,1485.0,121.0,15.36


In [7]:
df_yield.groupBy("Item").count().show()

+--------------------+-----+
|                Item|count|
+--------------------+-----+
|            Potatoes| 4276|
|             Sorghum| 3039|
|               Maize| 4121|
|Plantains and others|  556|
|            Soybeans| 3223|
|               Wheat| 3857|
|         Rice, paddy| 3388|
|                Yams|  847|
|      Sweet potatoes| 2890|
|             Cassava| 2045|
+--------------------+-----+



In [8]:
df2 = df_yield[df_yield.Item == "Potatoes"] ## Filter Item "Potatoes"

In [10]:
df2.count()

4276

In [11]:
df2.limit(10).toPandas()

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp
0,Albania,Potatoes,1990,66667,1485.0,121.00,16.37
1,Albania,Potatoes,1991,77818,1485.0,121.00,15.36
2,Albania,Potatoes,1992,82920,1485.0,121.00,16.06
3,Albania,Potatoes,1993,98446,1485.0,121.00,16.05
4,Albania,Potatoes,1994,81404,1485.0,201.00,16.96
5,Albania,Potatoes,1995,111323,1485.0,251.00,15.67
6,Albania,Potatoes,1996,106138,1485.0,313.96,15.64
7,Albania,Potatoes,1997,109874,1485.0,376.93,15.90
8,Albania,Potatoes,1998,127212,1485.0,439.89,16.27
9,Albania,Potatoes,1999,142018,1485.0,502.86,16.57


In [14]:
df2.columns[0]

'Area'

In [42]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import *

In [133]:
# Step 1: define columns as a features
assemble = VectorAssembler(
    inputCols = df2.columns[3:7],
    outputCol = "features"
)

In [134]:
train_data = assemble.transform(df2)

In [23]:
train_data.limit(10).toPandas()

,Area,Item,Year,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,features
0,Albania,Potatoes,1990,66667,1485.0,121.00,16.37,"[66667.0, 1485.0, 121.0, 16.37]"
1,Albania,Potatoes,1991,77818,1485.0,121.00,15.36,"[77818.0, 1485.0, 121.0, 15.36]"
2,Albania,Potatoes,1992,82920,1485.0,121.00,16.06,"[82920.0, 1485.0, 121.0, 16.06]"
3,Albania,Potatoes,1993,98446,1485.0,121.00,16.05,"[98446.0, 1485.0, 121.0, 16.05]"
4,Albania,Potatoes,1994,81404,1485.0,201.00,16.96,"[81404.0, 1485.0, 201.0, 16.96]"
5,Albania,Potatoes,1995,111323,1485.0,251.00,15.67,"[111323.0, 1485.0, 251.0, 15.67]"
6,Albania,Potatoes,1996,106138,1485.0,313.96,15.64,"[106138.0, 1485.0, 313.96, 15.64]"
7,Albania,Potatoes,1997,109874,1485.0,376.93,15.90,"[109874.0, 1485.0, 376.93, 15.9]"
8,Albania,Potatoes,1998,127212,1485.0,439.89,16.27,"[127212.0, 1485.0, 439.89, 16.27]"
9,Albania,Potatoes,1999,142018,1485.0,502.86,16.57,"[142018.0, 1485.0, 502.86, 16.57]"


In [129]:
#Step 2: set k-means
km = KMeans(k=3)
km.setSeed(2)
km.setWeightCol("Item")
km.setMaxIter(10)
km.getMaxIter()

10

In [135]:
#Step 3: Clustering
ouput = km.fit(train_data)

24/03/10 05:07:13 ERROR Instrumentation: org.apache.spark.sql.AnalysisException: Column 'Item' does not exist. Did you mean one of the following? [date, features, station_id, sensor_value3];
'Project [features#2864, unresolvedalias(UDF(cast('Item as double)), Some(org.apache.spark.sql.Column$$Lambda$4709/0x0000000101b8d040@13a20f83))]
+- Project [date#2560, sensor_value3#2538, station_id#2552, UDF(struct()) AS features#2864]
   +- Project [to_timestamp(date#2556, Some(yyyy-MM-dd HH:mm:ss), TimestampType, Some(Etc/UTC)) AS date#2560, sensor_value3#2538, station_id#2552]
      +- Project [cast(date#2548 as timestamp) AS date#2556, sensor_value3#2538, station_id#2552]
         +- Project [date#2548, sensor_value3#2538, cast(station_id#2543 as int) AS station_id#2552]
            +- Project [cast(date#2537 as timestamp) AS date#2548, sensor_value3#2538, station_id#2543]
               +- Project [date#2537, sensor_value3#2538, cast(station_id#2539 as int) AS station_id#2543]
              

AnalysisException: Column 'Item' does not exist. Did you mean one of the following? [date, features, station_id, sensor_value3];
'Project [features#2864, unresolvedalias(UDF(cast('Item as double)), Some(org.apache.spark.sql.Column$$Lambda$4709/0x0000000101b8d040@13a20f83))]
+- Project [date#2560, sensor_value3#2538, station_id#2552, UDF(struct()) AS features#2864]
   +- Project [to_timestamp(date#2556, Some(yyyy-MM-dd HH:mm:ss), TimestampType, Some(Etc/UTC)) AS date#2560, sensor_value3#2538, station_id#2552]
      +- Project [cast(date#2548 as timestamp) AS date#2556, sensor_value3#2538, station_id#2552]
         +- Project [date#2548, sensor_value3#2538, cast(station_id#2543 as int) AS station_id#2552]
            +- Project [cast(date#2537 as timestamp) AS date#2548, sensor_value3#2538, station_id#2543]
               +- Project [date#2537, sensor_value3#2538, cast(station_id#2539 as int) AS station_id#2543]
                  +- Relation [date#2537,sensor_value3#2538,station_id#2539]

In [ ]:
#Step 4: show output
output.summary.clusterSizes

In [38]:
output.clusterCenters()

NameError: name 'output' is not defined

In [24]:
!wget https://storage.googleapis.com/bd522_2021/data/sensor_12.csv
!wget https://storage.googleapis.com/bd522_2021/data/sensor_3.json

--2024-03-10 04:11:20--  https://storage.googleapis.com/bd522_2021/data/sensor_12.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.120.207, 142.250.103.207, 142.250.159.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.120.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62711541 (60M) [text/csv]
Saving to: ‘sensor_12.csv’

sensor_12.csv       100%[===================>]  59.81M  18.2MB/s    in 4.4s    

2024-03-10 04:11:25 (13.5 MB/s) - ‘sensor_12.csv’ saved [62711541/62711541]

--2024-03-10 04:11:26--  https://storage.googleapis.com/bd522_2021/data/sensor_3.json
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.120.207, 142.250.103.207, 142.250.159.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.120.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84182547 (80M) [application/json]
Saving to: ‘sensor_3.json’

sensor_3.json     

In [26]:
!hadoop fs -put sensor_12.csv /user/dataproc/sensor_12.csv
!hadoop fs -put sensor_3.json /user/dataproc/sensor_3.json

put: `/user/dataproc/sensor_12.csv': File exists
put: `/user/dataproc/sensor_3.json': File exists


In [27]:
!hadoop fs -ls /user/dataproc

Found 2 items
-rw-r--r--   2 root hadoop   62711541 2024-03-10 04:12 /user/dataproc/sensor_12.csv
-rw-r--r--   2 root hadoop   84182547 2024-03-10 04:12 /user/dataproc/sensor_3.json


In [94]:
df = spark.read.csv("/user/dataproc/sensor_12.csv",
                    sep=";" ,
                    inferSchema=True,
                    header=True
                   )

In [95]:
df2 = spark.read.json("/user/dataproc/sensor_3.json")

In [97]:
df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- sensor_value1: double (nullable = true)
 |-- station_id: integer (nullable = true)
 |-- sensor_value2: double (nullable = true)



In [98]:
# Data Preparation
# Convert df2 column to correct type
df2 = df2.withColumn("station_id", col('station_id').cast('integer'))
df2 = df2.withColumn("date", col('date').cast('timestamp'))
df2.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- sensor_value3: double (nullable = true)
 |-- station_id: integer (nullable = true)



In [99]:
df2 = df2.withColumn("date", to_timestamp(df2.date, "yyyy-MM-dd HH:mm:ss"))

In [100]:
df_123 = df2.join(df, ["station_id", "date"], "inner")

In [105]:
df_all = df_123.withColumn("date", to_timestamp(df_123.date, "yyyy-MM-dd HH:mm"))

In [106]:
df_all.limit(10).toPandas()

,station_id,date,sensor_value3,sensor_value1,sensor_value2
0,1,2018-01-01 00:00:00,98.758042,6.232350,9.291957
1,1,2018-01-01 00:02:00,67.183300,5.557534,8.492620
2,1,2018-01-01 00:07:00,14.692583,9.682279,14.459765
3,1,2018-01-01 00:15:00,41.435830,0.215575,0.437844
4,1,2018-01-01 00:18:00,65.574602,2.628964,4.040055
5,1,2018-01-01 00:24:00,45.228136,7.655527,11.504299
6,1,2018-01-01 00:27:00,92.232240,8.068260,12.135764
7,1,2018-01-01 00:28:00,60.473010,3.679941,5.540597
8,1,2018-01-01 00:30:00,63.099527,3.676593,5.597932
9,1,2018-01-01 00:32:00,9.420035,8.012445,11.938991


In [76]:
df_123.sort(df_123.station_id, df_123.date).limit(20).toPandas()

,station_id,date,sensor_value3,sensor_value3
0,1,2018-01-01 00:00:00,98.758042,98.758042
1,1,2018-01-01 00:01:00,83.311504,83.311504
2,1,2018-01-01 00:02:00,67.183300,67.183300
3,1,2018-01-01 00:03:00,96.351818,96.351818
4,1,2018-01-01 00:04:00,89.726888,89.726888
5,1,2018-01-01 00:05:00,45.478606,45.478606
6,1,2018-01-01 00:06:00,41.985771,41.985771
7,1,2018-01-01 00:07:00,14.692583,14.692583
8,1,2018-01-01 00:08:00,69.890227,69.890227
9,1,2018-01-01 00:09:00,50.039344,50.039344


In [50]:
# Transform all sensor value that lower than 0 to 0
df_missing = df_123.withColumn("sensor_value1",
                               when(df_123.sensor_value1 < 0, 0) \
                               .otherwise(df_123.sensor_value1)
                              ).withColumn("sensor_value2",
                               when(df_123.sensor_value2 < 0, 0) \
                               .otherwise(df_123.sensor_value2)
                              ) .withColumn("sensor_value3",
                               when(df_123.sensor_value3 < 0, 0) \
                               .otherwise(df_123.sensor_value3)
                              )

In [49]:
df_missing.summary().toPandas()

,summary,station_id,sensor_value1,sensor_value2,sensor_value3
0,count,1000000,1000000,1000000,1000000
1,mean,5.5,4.999532123608654,7.3990374746835865,49.993481785964626
2,stddev,2.8722827594107416,2.8867280999458202,4.347463698984681,28.881502410793132
3,min,1,0.0,0.0,0.0
4,25%,3,2.4970664977442065,3.656034027145632,24.99245448966307
5,50%,5,4.999426406861214,7.320367186549411,49.99004473906103
6,75%,8,7.4963847645179635,10.979495732938485,75.00635531644248
7,max,10,10.16457616320874,17.114699127085327,100.13130454490204


In [56]:
df_all = df_123.withColumn("sensor_AVG",(col("sensor_value1") + col("sensor_value2") + col("sensor_value3"))/3)

In [58]:
df_all = df_all.select("station_id", "date", "sensor_AVG")

In [64]:
df_all.sort("date").limit(10).toPandas()

,station_id,date,sensor_AVG
0,6,2018-01-01,38.165278
1,3,2018-01-01,24.125779
2,9,2018-01-01,29.330350
3,8,2018-01-01,11.150519
4,10,2018-01-01,14.357048
5,7,2018-01-01,38.056310
6,1,2018-01-01,38.094116
7,4,2018-01-01,14.775205
8,2,2018-01-01,18.730295
9,5,2018-01-01,24.474046


In [108]:
train_data_sensor = df_all.groupBy("date").avg("sensor_value1", "sensor_value2", "sensor_value3")

In [119]:
assembler = VectorAssembler(
    inputCols=["avg(sensor_value1)", "avg(sensor_value2)", "avg(sensor_value3)"],
    outputCol="features"
)

train_data_with_features = assembler.transform(train_data_sensor)

train_data_with_features.limit(10).toPandas()

,date,avg(sensor_value1),avg(sensor_value2),avg(sensor_value3),features
0,2018-01-01 08:40:00,5.577034,8.410761,45.329286,"[5.577034191498354, 8.410761256544216, 45.3292..."
1,2018-01-01 09:38:00,4.670715,6.710940,48.550329,"[4.6707150421721195, 6.710939758210847, 48.550..."
2,2018-01-02 09:25:00,5.424913,7.829558,48.086506,"[5.42491279591634, 7.829557672221871, 48.08650..."
3,2018-01-03 00:17:00,6.276174,9.336096,51.850831,"[6.276173550873747, 9.336096400544678, 51.8508..."
4,2018-01-05 05:50:00,5.540920,8.160996,45.114480,"[5.540919642351772, 8.160996279330561, 45.1144..."
5,2018-01-03 10:52:00,4.903428,7.238839,41.668264,"[4.90342799014173, 7.238838521050797, 41.66826..."
6,2018-01-03 14:25:00,4.379394,6.315839,67.850950,"[4.379394100321763, 6.315839209064323, 67.8509..."
7,2018-01-04 03:04:00,4.007301,5.716310,37.205001,"[4.00730065250619, 5.716310194210995, 37.20500..."
8,2018-01-04 03:44:00,6.179993,9.044403,41.323637,"[6.179993402527933, 9.044403484745457, 41.3236..."
9,2018-01-05 03:14:00,3.745705,5.375071,58.365894,"[3.7457049444761084, 5.375070675045524, 58.365..."


In [ ]:
kMeans_Sensor = KMeans(k=5, seed=1, maxIter=5)

In [116]:
kMeans_Sensor.setWeightCol("date")
kMeans_Sensor.getMaxIter()

5

In [117]:
#Step 3: Clustering
output = kMeans_Sensor.fit(train_data_with_features)

In [121]:
#Step 4: show output
output.summary.clusterSizes

[30829, 17615, 16546, 29090, 5920]

In [123]:
centers = output.clusterCenters()
centers

[array([ 5.00625094,  7.40870835, 45.66761629]),
 array([ 5.00242511,  7.40364729, 60.21075404]),
 array([ 5.00536296,  7.40769872, 36.33459563]),
 array([ 4.98949563,  7.38436279, 53.15782149]),
 array([ 4.9864894 ,  7.38011081, 68.26669173])]

In [128]:
print("Cluster Centers: ")
i = 1;
for center in centers:
    print(f"center[{i}] ==> {center}")
    i = i+1

Cluster Centers: 
center[1] ==> [ 5.00625094  7.40870835 45.66761629]
center[2] ==> [ 5.00242511  7.40364729 60.21075404]
center[3] ==> [ 5.00536296  7.40769872 36.33459563]
center[4] ==> [ 4.98949563  7.38436279 53.15782149]
center[5] ==> [ 4.9864894   7.38011081 68.26669173]
